Install dependencies

In [12]:
!source venv/bin/activate
!pip install kaggle
!pip install kagglehub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=36d20d3bd5e269e329d6d1696a42254293b4146b5f57a43d722f87299a10000e
  Stored in directory: /Users/tunguyen/Library/Caches/pip/wheels/9f/af/22/bf406f913dc7506a485e60dce8143741abd0a92a19337d83a3
Successfully built kaggle


In [23]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()

dataset_path = 'datasets/datamunge/sign-language-mnist'
api.dataset_download_files("datamunge/sign-language-mnist", path=dataset_path, unzip=True)
print("Dataset has been downloaded into ", dataset_path)

Dataset URL: https://www.kaggle.com/datasets/datamunge/sign-language-mnist
Dataset has been downloaded into  datasets/datamunge/sign-language-mnist
